In [9]:
import os
import json
import pandas as pd
import traceback

In [10]:
from langchain_groq import ChatGroq

In [13]:
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
KEY = os.getenv("GROQ_API_KEY")

In [7]:
llm = ChatGroq(temperature=0.5,groq_api_key=KEY,model='mixtral-8x7b-32768')